# Data Anaysis Project part1:

I am using Vancouver rental ads on craigslist to analyze rental market. Later, I will create the class to be able to analyze dynamically. Also, I will start collecting the data and save them to the CSV, so I can analyze the trend of the rental market as well. 

I will follow what I am learning from Google Data Analytics course to complete this project.

* Ask
* Prepare
* Process
* Analyze
* Share
* Act



## Overview of the project:

Analyze housing rental market in Fraser Valley to find out what is hot to real estate investors in this area. Iam going to create the code to analyze dynamically so the investors can apply in their area for their own use.  


## ASK 
    
    Find out what is 'HOT' in the market for investor, so i can find a property for them. Find out supply and demand of the rental housing market and calculate appropreate purchase price. 

#### What are the questions to ask?
Supply side:
* How much is the market rental price?
* What type of the rental is most popular and least popular?
* How many ads per day? Number of supply 
* Is rental price rising or decreasing over time?
* What is the correlation between housing price and rental price?
* From those questions, what is my prediction of the rental market? 

Demand side:
* What kind of propety does people look for rent?
* What is the price range they are looking for?
* What are the purposes of renting. (Who is looking for rent)





## Prepare  

#### Using Craigslist add to collect market data: (Supply side)
Scrape craigslist rental housing pages and store in csv file. 

<a href='https://github.com/rileypredum/East-Bay-Housing-Web-Scrape/blob/master/EB_Apt_Prices_Final.ipynb'>Code: I will use this code and modify for my situation.  </a> <br>
<a href='https://towardsdatascience.com/web-scraping-craigslist-a-complete-tutorial-c41cea4f4981'>Explaning</a>

## Process

I noticed that the all the values are string, so I will convert to correct data type as I collect the data. 



In [5]:
# Import libralis
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.display import clear_output

In [6]:
class Url:
    def __init__(self, city_name, sub_city=None, num_post=0):
        self.city_name = city_name
        self.sub_city = sub_city
        self.num_post = num_post
        self.url = self.get_url()
        
    def get_url(self):
        if self.city_name == 'Abbotsford':
            url = f'https://{self.city_name.lower()}.craigslist.org/search/apa?s={self.num_post}sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates'
        else:
            url = f'https://{self.city_name.lower()}.craigslist.org/search/{self.sub_city}/apa?s={self.num_post}sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates'
        return url 

In [7]:
# Vanvouver rental housing
url = Url("Vancouver", 'van')
url_ = url.url

response = get(url_)


soup = BeautifulSoup(response.text, 'html.parser')

posts = soup.find_all('li', class_='result-row')
print(type(posts)) # to double check that I got resultSet
print(len(posts)) # to double check I got 120 elements

<class 'bs4.element.ResultSet'>
126


In [8]:
%%time
# Find the total number of posts to find the limit of the pagination
city = 'Vancouver'
response = get(url_)
soup = BeautifulSoup(response.text, 'html.parser')
posts = soup.find_all('li', class_='result-row')
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
print(f"Total number of search result: {results_total}")

# each page has 119 posts so each new page is defined as follows; s=120, s=240, .....
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_date = []
post_cities = []
num_bedroom = []
sqfts = []
prices = []
post_titles = []
post_links = []

for page in pages:
    
    # get request
    url = Url(city_name=city, sub_city='van', num_post=page)
    url_ = url.url
    response = get(url_)
    
    sleep(1)
    
    # throw warning for status code that are not 200
    if response.status_code != 200:
        warn(f'Request: {requests}; Status code: {response.status_code}')
        
    soup = BeautifulSoup(response.text, 'html.parser')
    
    posts = soup.find_all('li', class_='result-row')
    
    
    # Extract data 
    for post in posts:
        
        if post.find('span', class_ = 'result-hood') is not None:
            
            # Posting date
            post_datetime = post.find('time', class_='result-date')['datetime']
            post_date.append(post_datetime)
            
            # Neighbourhoods
            post_city = post.find('span', class_='result-hood').text.strip('( )')
            post_cities.append(post_city)
            
            # title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_titles.append(post_title_text)
            
            # Post price in integer
            price = int(post.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
            prices.append(price)
            
            # Post link
            post_link = post_title['href']
            post_links.append(post_link)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                num_bedroom.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)
            #    num_bedroom.append(bedroom_count)
                
            #    sqft = np.nan
            #    sqfts.append(sqft)
        iterations += 1
        #print("Page" + str(iterations) + "Scraped Successfully!")   

Total number of search result: 3000
CPU times: total: 4.8 s
Wall time: 40.1 s


In [23]:
# create DataFrame
df = pd.DataFrame({'Post Datetime': post_date,
                   'Post Title': post_titles,
                   'Post URL': post_links,
                  'Neighborhood': post_cities,
                  'Bedroom': num_bedroom,
                  'SQFT': sqfts,
                  'Price': prices})

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3052 entries, 0 to 3051
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3052 non-null   object 
 1   Post Title     3052 non-null   object 
 2   Post URL       3052 non-null   object 
 3   Neighborhood   3052 non-null   object 
 4   Bedroom        2860 non-null   object 
 5   SQFT           2680 non-null   float64
 6   Price          3052 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 167.0+ KB
None


,Post Datetime,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
0,2022-12-10 17:41,WATER VIEW! Penthouse with high-end furniture,https://vancouver.craigslist.org/van/apa/d/van...,"Vancouver - VGH, Fairview, Granville and W 8th...",2,993.0,3950
1,2022-12-10 17:39,"Pet Friendly, 2 Bed, 1 Bath, Balcony, Parking,...",https://vancouver.craigslist.org/van/apa/d/van...,"Vancouver, Olympic Village, False Creek, Cross...",2,792.0,3150
2,2022-12-10 17:39,"Upper Floor of House, 3 Bedroom, 2 Bathroom, L...",https://vancouver.craigslist.org/van/apa/d/van...,"East Vancouver, Renfrew, Kensington Cedar Cottage",3,1061.0,3100
3,2022-12-10 17:39,"Outdoor patio and BBQ area, LVP flooring, 2 el...",https://vancouver.craigslist.org/van/apa/d/van...,"1022 Nelson Street, Vancouver, BC",2,691.0,3800
4,2022-12-10 17:33,Prosprise Realty* 2 bed 1 bath Lower suite for...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,800.0,2200


## Process

Check the dataset I created from the Craigslist ad and final adjustment of the dataset to analyze the data. 

First to check the dataset:

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3052 entries, 0 to 3051
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3052 non-null   object 
 1   Post Title     3052 non-null   object 
 2   Post URL       3052 non-null   object 
 3   Neighborhood   3052 non-null   object 
 4   Bedroom        2860 non-null   object 
 5   SQFT           2680 non-null   float64
 6   Price          3052 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 167.0+ KB


Convert Post Datetime object to Datetime

I would like to distingish FULLY FURNISHED suite and not furnished suite. I will create another column to show if it isTURE or FALSE. I noticed the title says if it is furnished suite or not (If not on the title, I assume that the suite is not furnished) 

Also, I noticed that the number of bedroom is string, so I will convert to integer. 

In [25]:
# Convert Datetime then split seperate columns
df['Post Datetime'] = pd.to_datetime(df["Post Datetime"])
df['Post Date'] = df['Post Datetime'].dt.date
df['Post Time'] = df['Post Datetime'].dt.time
df = df[['Post Datetime', 'Post Date', 'Post Time', 'Post Title', 'Post URL', 'Neighborhood', 'Bedroom', 'SQFT', 'Price']]
df.head()

,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
0,2022-12-10 17:41:00,2022-12-10,17:41:00,WATER VIEW! Penthouse with high-end furniture,https://vancouver.craigslist.org/van/apa/d/van...,"Vancouver - VGH, Fairview, Granville and W 8th...",2,993.0,3950
1,2022-12-10 17:39:00,2022-12-10,17:39:00,"Pet Friendly, 2 Bed, 1 Bath, Balcony, Parking,...",https://vancouver.craigslist.org/van/apa/d/van...,"Vancouver, Olympic Village, False Creek, Cross...",2,792.0,3150
2,2022-12-10 17:39:00,2022-12-10,17:39:00,"Upper Floor of House, 3 Bedroom, 2 Bathroom, L...",https://vancouver.craigslist.org/van/apa/d/van...,"East Vancouver, Renfrew, Kensington Cedar Cottage",3,1061.0,3100
3,2022-12-10 17:39:00,2022-12-10,17:39:00,"Outdoor patio and BBQ area, LVP flooring, 2 el...",https://vancouver.craigslist.org/van/apa/d/van...,"1022 Nelson Street, Vancouver, BC",2,691.0,3800
4,2022-12-10 17:33:00,2022-12-10,17:33:00,Prosprise Realty* 2 bed 1 bath Lower suite for...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,800.0,2200


In [26]:
# Convert Bedroom number strings to Integer then NaN value to 0 assume 0 bedroom is studio appartment. 
df["Bedroom"] = pd.to_numeric(df["Bedroom"], errors='ignore').astype('Int64')
df["Bedroom"] = df["Bedroom"].fillna(0)

In [27]:
%%time
# Check if word 'furnished' in the title post. 
search_word = 'furnished'
furnished = []
for i in range(len(df)):
    post_title_list = df['Post Title'][i].lower().split(" ")
    if any(word == search_word for word in post_title_list):
        furnished.append(True)
    else:
        furnished.append(False)
df["Furnished?"] = furnished

CPU times: total: 0 ns
Wall time: 16 ms


In [30]:
# Remove duplicates
df.drop_duplicates(inplace=True)
len(df)

3005

## Analyze

In [31]:
# Count how many rentals are furnished 
num_furnished_suite = df[df['Furnished?'] == True]["Furnished?"].count()
num_unfurnished_suite = len(df) - num_furnished_suite
furnished_suite_percent = np.round((num_furnished_suite / len(df)) * 100, 2)
print(f"Number of furnished suite is {num_furnished_suite}, and it's {furnished_suite_percent}% of the entire list.")

Number of furnished suite is 652, and it's 21.7% of the entire list.


In [32]:
# Find out the number of ads on each bedrooms number.  
num_bed = np.sort(df["Bedroom"].unique())
suite_counts = []
for i in range(len(num_bed)):
    suite_count = len(df[df['Bedroom'] == num_bed[i]])
    suite_counts.append(suite_count)
    #print(f'Number of {num_bed[i]} Bedroom suite: {len(df[df["Bedroom"] == num_bed[i]])}')
#print(f"List of suite counts: {suite_counts}")

In [33]:
# Find out Mean, Max, Min of SQFT and Price of each bedroom and SUM of furnished suite and total suite on each bedroom. 
df_count = df.groupby(by=['Bedroom']).agg({'SQFT': ['mean', 'max', 'min'],
                                                 'Price': ['mean', 'max', 'min'],
                                                  'Furnished?': ['sum', 'count']})
df_count['percent furnished'] = df_count['Furnished?']['sum'] / df_count["Furnished?"]['count']
df_count

SQFT                         Price               Furnished?  \
                mean      max     min         mean     max   min        sum   
Bedroom                                                                       
0         436.604651   1357.0   103.0  2273.239583    8000   840         58   
1         621.118172   2000.0     1.0  2527.156900   10000   100        271   
2         996.846154   2936.0   300.0  4339.439778  500000   700        244   
3        1570.875912   4000.0    14.0  5554.560656   24995  1250         45   
4        2595.505747   6800.0   900.0  6609.718750   25000   950         14   
5        3553.600000   6500.0  1500.0  8111.148148   19500  3300         12   
6        6349.739130  29789.0  1811.0  9252.200000   22880   950          6   
7        3920.285714   6000.0  1300.0  9511.250000   19995  1300          2   
8        4511.500000   4980.0  3106.0  7840.000000    9000  6500          0   

              percent furnished  
        count                    
Bedroom                          
0         192          0.302083  
1        1058          0.256144  
2        1262          0.193344  
3         305          0.147541  
4          96          0.145833  
5          54          0.222222  
6          25          0.240000  
7           8          0.250000  
8           5          0.000000

In [34]:
# Price per furnished suit on each bedroom
#df_price_furnished = df.drop(['Post Datetime', 'Post Title', 'Post URL', 'Neighborhood'], axis=1)
df_price_furnished = df.groupby(['Bedroom', 'Furnished?']).agg({'Price': ['mean', 'max', 'min', 'count'],
                                                               "SQFT": ['mean', 'max', 'min']})
df_price_furnished

Price                              SQFT           \
                            mean     max   min count          mean      max   
Bedroom Furnished?                                                            
0       False        2208.641791    4050   950   134    439.697479   1200.0   
        True         2422.482759    8000   840    58    429.660377   1357.0   
1       False        2382.705210    6500   120   787    619.449011   2000.0   
        True         2946.653137   10000   100   271    625.687500   1855.0   
2       False        3746.887033   18000  1000  1018    976.068736   2936.0   
        True         6811.647541  500000   700   244   1078.685590   2873.0   
3       False        5196.984615   20000  1250   260   1539.869565   4000.0   
        True         7620.555556   24995  2700    45   1732.954545   3800.0   
4       False        6153.634146   25000   950    82   2537.918919   4600.0   
        True         9281.071429   18500  4800    14   2923.307692   6800.0   
5       False        7515.095238   19500  3300    42   3513.842105   6426.0   
        True        10197.333333   17995  6088    12   3679.500000   6500.0   
6       False        8306.578947   16500   950    19   4939.777778  10098.0   
        True        12246.666667   22880  5600     6  11425.600000  29789.0   
7       False        7699.166667    9995  1300     6   3634.000000   4800.0   
        True        14947.500000   19995  9900     2   4636.000000   6000.0   
8       False        7840.000000    9000  6500     5   4511.500000   4980.0   

                            
                       min  
Bedroom Furnished?          
0       False        103.0  
        True         180.0  
1       False          1.0  
        True         400.0  
2       False        300.0  
        True         550.0  
3       False         14.0  
        True         900.0  
4       False        900.0  
        True        2000.0  
5       False       1500.0  
        True        2250.0  
6       False       1811.0  
        True        4639.0  
7       False       1300.0  
        True        3272.0  
8       False       3106.0

Dropped extrimely low price add, and it seems no outlier on the price

In [35]:
# Count how many ads per day. 
df.groupby('Post Date').agg({'Post Title': 'count'})

,Post Title
Post Date,
2022-11-17,1
2022-11-19,1
2022-11-20,1
2022-11-21,14
2022-11-22,75
2022-11-23,63
2022-11-24,41
2022-11-25,45
2022-11-26,25


### Create dataFrame for just one day

In [36]:
# December 7, 2022's ads
df_one_day = df[df['Post Date'] == datetime(2022, 12, 9).date()]
df_one_day

,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?
332,2022-12-09 23:57:00,2022-12-09,23:57:00,3BR BASEMENT SUITE,https://vancouver.craigslist.org/van/apa/d/van...,Killarney,3,NaN,2500,False
333,2022-12-09 23:56:00,2022-12-09,23:56:00,2bd+2bth+pkg > METROLIVING > Perfect Downtown ...,https://vancouver.craigslist.org/van/apa/d/van...,Excellent Floor Plan TWO BED TWO BATH w/PARKIN...,2,931.0,3300,False
334,2022-12-09 23:55:00,2022-12-09,23:55:00,2 bed 2 bath laneway house for rent,https://vancouver.craigslist.org/van/apa/d/van...,vancouver,2,780.0,2800,False
335,2022-12-09 23:47:00,2022-12-09,23:47:00,Well maintained 2BED+2BATH CONDO In Downtown Y...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,900.0,3300,False
336,2022-12-09 23:38:00,2022-12-09,23:38:00,Coal Harbour Work or Live/Work luxury Suite,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,0,817.0,3600,False
...,...,...,...,...,...,...,...,...,...,...
922,2022-12-09 00:15:00,2022-12-09,00:15:00,Kitsilano - 2BR available January 25th,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,NaN,2100,False
923,2022-12-09 00:14:00,2022-12-09,00:14:00,2 Br 2 Bath with den and flex room,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,1100.0,3600,False
924,2022-12-09 00:11:00,2022-12-09,00:11:00,Wonderful 2 bed/2 bath unit with storage and p...,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,2,815.0,3485,False
925,2022-12-09 00:02:00,2022-12-09,00:02:00,Gorgeous Ocean Views One Bedroom Plus Den Furn...,https://vancouver.craigslist.org/van/apa/d/van...,Coal Harbour,1,670.0,3350,True


## Share

## Act